# 推荐系统 & 点击率预估

Event Recommendation Engine Challenge 是Kaggle 上的一个推荐任务竞赛，根据用户的历史参加活动记录、用户的社交信息、以及用户历史上在App上浏览和点击的活动，预测用户是否会对某个活动感兴趣。  

以下代码实现内容：
1. 实现基于用户的协同过滤（用户本身，用户好友分析）  
2. 实现基于活动的协同过滤（活动本身，用户活动关联分析）  
3. 能够实现基于模型的协同过滤（矩阵分解/LFM）  
4. 组合各种前面生成的特征，生成新的训练数据，并对数据进行保存  

In [1]:
from __future__ import division

import pickle
import numpy as np
import scipy.io as sio
import scipy.sparse as ss
from numpy.random import random  
from collections import defaultdict

导入必要工具包（精确除法、稀疏矩阵处理等）

## 计算相似度
用户之间相似度：首先需要用数学的方法表示用户，即用向量，向量的内容则是该用户对不同活动的评价，使用不同用户的向量来计算相似度  
活动之间相似度：首先需要用数学的方法表示活动，即用向量，向量的内容则是不同用户对该活动评价，使用不同活动的向量来计算相似度  

相似度取值：[-1,1]

## 计算用户之间的相似度矩阵，便于后续直接查询

下方计算基于用户的协同过滤时，需要频繁使用到用户之间的相似度，为了减少计算量，可以提前将所有用户的相似度计算好并存储在矩阵中，后续直接在矩阵中查询取值就行

In [3]:
# 导入用户和活动的重编码表（前期已进行处理，只取训练集和测试集出现过的用户和活动）
userIndex = pickle.load(open("PE_userIndex.pkl", 'rb'))
eventIndex = pickle.load(open("PE_eventIndex.pkl", 'rb'))
n_users = len(userIndex)
n_events = len(eventIndex)

# 导入关联用户对和分数矩阵
uniqueUserPairs = pickle.load(open("FE_uniqueUserPairs.pkl", 'rb'))
userEventScores = sio.mmread("PE_userEventScores").todense()
print(userEventScores)
print(type(userEventScores))                  # 验证评分矩阵为稀疏矩阵 [3391, 13418]
userEventScores = np.mat(userEventScores)     # np.mat作用是转化为np中定义的稀疏矩阵，可能不影响，但总觉得统一格式好点
#print(userEventScores)
#print(type(userEventScores))                 # 打印结果和未转化前一致

mean_score = np.mean(userEventScores,axis=1)  # 相似度矩阵中需要计算每个user打分的均值，提前计算并得到矩阵形式 [1，13428]
print(mean_score)
print(mean_score.shape)                       # 得到 [3391,1] 维数的均值矩阵

row_userEventScores = userEventScores - mean_score    # 计算每行的偏差（单个event的分数减均值）
row_userEventScores = np.array(row_userEventScores)
# 不转化为array，下面相乘时提示元素没有对齐，可能矩阵相乘时我没有转置，两个都是行矩阵
# 转换为array就不会提示，直接对应相乘


# 建立存储用户相似度的矩阵，对角线元素为1.0
userSimMatrix_userBased= ss.dok_matrix((n_users, n_users))
for i in range(n_users):
    userSimMatrix_userBased[i,i] = 1.0
    
    
# 取出关联用户对应的活动分数，进行相似度计算，公式中直接用矩阵和向量的方式计算，比轮询计算单个数值快
# 此处计算相似度使用 Pearson相关系数公式，公式形式有多种，前面代码计算基本用户本身属性的用户间相似度使用的是另一种形式
for u1, u2 in uniqueUserPairs:
    i = u1
    j = u2
    if(i,j) not in userSimMatrix_userBased.keys():
        sim = sum(row_userEventScores[i]*row_userEventScores[j])\
             /(np.sqrt(sum(np.power(row_userEventScores[i],2))))\
             /(np.sqrt(sum(np.power(row_userEventScores[j],2))))
        #print(sim)
        userSimMatrix_userBased[i,j] = sim
        userSimMatrix_userBased[j,i] = sim
sio.mmwrite('userSimMatrix_userBased',userSimMatrix_userBased)


[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
<class 'numpy.matrixlib.defmatrix.matrix'>
[[0.00000000e+00]
 [0.00000000e+00]
 [1.49053510e-04]
 ...
 [0.00000000e+00]
 [0.00000000e+00]
 [7.45267551e-05]]
(3391, 1)


In [4]:
print(row_userEventScores[i])
row_userEventScores[i].shape

[-0.00037263 -0.00037263 -0.00037263 ... -0.00037263 -0.00037263
 -0.00037263]


(13418,)

In [5]:
userEventScores.shape

(3391, 13418)

In [6]:
print(userSimMatrix_userBased)
userSimMatrix_userBased.shape

  (0, 0)	1.0
  (1, 1)	1.0
  (2, 2)	1.0
  (3, 3)	1.0
  (4, 4)	1.0
  (5, 5)	1.0
  (6, 6)	1.0
  (7, 7)	1.0
  (8, 8)	1.0
  (9, 9)	1.0
  (10, 10)	1.0
  (11, 11)	1.0
  (12, 12)	1.0
  (13, 13)	1.0
  (14, 14)	1.0
  (15, 15)	1.0
  (16, 16)	1.0
  (17, 17)	1.0
  (18, 18)	1.0
  (19, 19)	1.0
  (20, 20)	1.0
  (21, 21)	1.0
  (22, 22)	1.0
  (23, 23)	1.0
  (24, 24)	1.0
  :	:
  (2908, 3274)	-7.45323097563392e-05
  (1869, 1515)	0.6545560631824034
  (1515, 1869)	0.6545560631824034
  (1051, 1280)	-0.00010540853153624303
  (1280, 1051)	-0.00010540853153624303
  (3083, 1610)	-0.0002582356120275864
  (1610, 3083)	-0.0002582356120275864
  (1235, 435)	-0.0001825865412756965
  (435, 1235)	-0.0001825865412756965
  (1092, 360)	-0.00012910337003435858
  (360, 1092)	-0.00012910337003435858
  (317, 1130)	0.4999440976427195
  (1130, 317)	0.4999440976427195
  (3081, 2697)	0.5773072363368542
  (2697, 3081)	0.5773072363368542
  (211, 987)	-0.00010540853153622814
  (987, 211)	-0.00010540853153622814
  (1678, 3386)	-0.0003

(3391, 3391)

可以看出得出的相似度矩阵也很稀疏，而相似系数也比较合理，其中出现负值可能是因为两个用户之间的行为刚好相反或很不一样

## 计算活动之间的相似度矩阵，便于后续直接查询

活动之间的相似度计算和用户相似度计算有点类似，上面的一些数据可以直接使用（如用户打分相对均值的偏差等）

In [7]:
# 导入关联活动对
uniqueEventPairs = pickle.load(open("PE_uniqueEventPairs.pkl", 'rb'))

# 建立存储活动相似度的矩阵，对角线元素为1.0
eventSimMatrix_eventBased = ss.dok_matrix((n_events, n_events))
for i in range(n_events):
    eventSimMatrix_eventBased[i,i] = 1.0

# 取出关联活动对应的分数，进行相似度计算   维数 [13418， 13418]
# 使用夹角余弦公式计算
for event1, event2 in uniqueUserPairs:
    m = event1
    n = event2
    if(m,n) not in eventSimMatrix_eventBased.keys():
        sim_e = sum(row_userEventScores[:,m]*row_userEventScores[:,n])\
             /(np.sqrt(sum(np.power(row_userEventScores[:,m],2))))\
             /(np.sqrt(sum(np.power(row_userEventScores[:,n],2))))
        #print(sim)
        eventSimMatrix_eventBased[m,n] = sim_e
        eventSimMatrix_eventBased[n,m] = sim_e
sio.mmwrite('eventSimMatrix_eventBased',eventSimMatrix_eventBased)

In [8]:
print(row_userEventScores[:,m])
row_userEventScores[:,m].shape

[ 0.00000000e+00  0.00000000e+00 -1.49053510e-04 ...  0.00000000e+00
  0.00000000e+00 -7.45267551e-05]


(3391,)

In [9]:
print(eventSimMatrix_eventBased)
eventSimMatrix_eventBased.shape

  (0, 0)	1.0
  (1, 1)	1.0
  (2, 2)	1.0
  (3, 3)	1.0
  (4, 4)	1.0
  (5, 5)	1.0
  (6, 6)	1.0
  (7, 7)	1.0
  (8, 8)	1.0
  (9, 9)	1.0
  (10, 10)	1.0
  (11, 11)	1.0
  (12, 12)	1.0
  (13, 13)	1.0
  (14, 14)	1.0
  (15, 15)	1.0
  (16, 16)	1.0
  (17, 17)	1.0
  (18, 18)	1.0
  (19, 19)	1.0
  (20, 20)	1.0
  (21, 21)	1.0
  (22, 22)	1.0
  (23, 23)	1.0
  (24, 24)	1.0
  :	:
  (2908, 3274)	0.9999999999999998
  (1869, 1515)	0.9999999999999998
  (1515, 1869)	0.9999999999999998
  (1051, 1280)	0.9999999999999998
  (1280, 1051)	0.9999999999999998
  (3083, 1610)	0.9999999999999998
  (1610, 3083)	0.9999999999999998
  (1235, 435)	-0.04174281594577362
  (435, 1235)	-0.04174281594577362
  (1092, 360)	0.9999999999999998
  (360, 1092)	0.9999999999999998
  (317, 1130)	0.9999999999999998
  (1130, 317)	0.9999999999999998
  (3081, 2697)	0.9999999999999998
  (2697, 3081)	0.9999999999999998
  (211, 987)	-0.05383239994357515
  (987, 211)	-0.05383239994357515
  (1678, 3386)	0.0015251141537247846
  (3386, 1678)	0.001525114

(13418, 13418)

In [10]:
# 将所有特征串联起来，构成RS_Train.csv
#RS_Test.csv
#为最后推荐系统做准备


class RecommonderSystem:
  def __init__(self):
    # 读入数据做初始化
    
    #用户和活动新的索引
    self.userIndex = pickle.load(open("PE_userIndex.pkl", 'rb'))
    self.eventIndex = pickle.load(open("PE_eventIndex.pkl", 'rb'))
    self.n_users = len(self.userIndex)
    self.n_items = len(self.eventIndex)
    
    #用户-活动关系矩阵R
    self.userEventScores = sio.mmread("PE_userEventScores").todense()
    
    #倒排表
    ##每个用户参加的事件
    self.itemsForUser = pickle.load(open("PE_eventsForUser.pkl", 'rb'))
    ##事件参加的用户
    self.usersForItem = pickle.load(open("PE_usersForEvent.pkl", 'rb'))
    
    #基于模型的协同过滤参数初始化,训练
    self.init_SVD()
    self.train_SVD(trainfile = "train.csv")
    
    #根据用户属性计算出的用户之间的相似度
    self.userSimMatrix = sio.mmread("US_userSimMatrix").todense()
    
    #根据活动属性计算出的活动之间的相似度
    self.eventPropSim = sio.mmread("EV_eventPropSim").todense()
    self.eventContSim = sio.mmread("EV_eventContSim").todense()
    
    #每个用户的朋友的数目
    self.numFriends = sio.mmread("UF_numFriends")
    #用户的每个朋友参加活动的分数对该用户的影响
    self.userFriends = sio.mmread("UF_userFriends").todense()
    
    #活动本身的热度
    self.eventPopularity = sio.mmread("EA_eventPopularity").todense()
    
    # 先读取两个相似度的矩阵，存在变量缓存中，不用每次调用函数都读一次，这样很慢
    self.userSimMatrix_users = sio.mmread("userSimMatrix_userBased").todense()
    self.userSimMatrix_events = sio.mmread("eventSimMatrix_eventBased").todense()
    
  def init_SVD(self, K=20):
    #初始化模型参数（for 基于模型的协同过滤SVD_CF）
    self.K = K                       # K一般取值为20 -- 100，此处取20
    
    #init parameters
    #bias
    self.bi = np.zeros(self.n_items)  # 记录活动本身的特性，作为偏差
    self.bu = np.zeros(self.n_users)  # 记录用户本身的特性，作为偏差
    
    #the small matrix
    self.P = random((self.n_users,self.K))/10*(np.sqrt(self.K))
    self.Q = random((self.K, self.n_items))/10*(np.sqrt(self.K))  
                  
          
  def train_SVD(self,trainfile = 'train.csv', steps=100,gamma=0.04,Lambda=0.15):
    #训练SVD模型（for 基于模型的协同过滤SVD_CF）
    #gamma：为学习率
    #Lambda：正则参数
    
    #偷懒了，为了和原来的代码的输入接口一样，直接从训练文件中去读取数据
    print ("SVD Train...")
    ftrain = open(trainfile, 'r')
    ftrain.readline()
    self.mu = 0.0
    n_records = 0
    uids = []  #每条记录的用户索引
    i_ids = [] #每条记录的item索引
    #用户-Item关系矩阵R（内容同userEventScores相同），临时变量，训练完了R不再需要
    R = np.zeros((self.n_users, self.n_items))
    
    for line in ftrain:
        cols = line.strip().split(",")
        u = self.userIndex[cols[0]]  #用户
        i = self.eventIndex[cols[1]] #活动
        
        uids.append(u)         # 得出训练集中的用户集合
        i_ids.append(i)        # 得出训练集中的活动集合
        
        R[u,i] = int(cols[4])  #interested  分数
        self.mu += R[u,i]
        n_records += 1
    
    ftrain.close()
    self.mu /= n_records       #计算所有打分的均值，公式中需使用
    
    # 我感觉真正在做模型训练时，学习率和正则参数应该都是超参数，需要调参的，并且这两个参数非常重要，学习率一般采用
    # 随迭代次数增加逐渐减小的方法，既能在前期快速收敛，也能在训练后期防止overshot
    # steps应该是终止条件，终止条件可以是迭代次数小于某个值，预测误差小于某个阈值，梯度下降幅度小于某个阈值
    
    
    # 各个参数已经有了初始值
    # 目标函数SSE = 预测残差的平方 + P的L2正则 + Q的L2正则 + bi的L2正则 + bu的L2正则

    for n_round in range(steps):
        for n_count in range(n_records):
            pred_score = self.pred_SVD(uids[n_count],i_ids[n_count])
            e_ui = R[uids[n_count], i_ids[n_count]] - pred_score
            
            # 求目标函数中各参数的偏导数
            gSSE_gPuk = -e_ui * self.Q[:,i_ids[n_count]] + Lambda * self.P[uids[n_count],:] #对 Puk 求偏导，维数 K
            gSSE_gQki = -e_ui * self.P[uids[n_count],:] + Lambda * self.Q[:,i_ids[n_count]] #对 Qki 求偏导，维数 K
            gSSE_gBu = -e_ui + Lambda * self.bu[uids[n_count]]                              #对 Bu 求偏导，维数 1
            gSSE_gBi = -e_ui + Lambda * self.bi[i_ids[n_count]]                             #对 Bi 求偏导，维数 1
            
            # 更新迭代参数,每个参数都是负梯度方向下降
            self.P[uids[n_count],:] = self.P[uids[n_count],:] - gamma * gSSE_gPuk
            self.Q[:,i_ids[n_count]] = self.Q[:,i_ids[n_count]] - gamma * gSSE_gQki
            self.bu[uids[n_count]] = self.bu[uids[n_count]] - gamma * gSSE_gBu
            self.bi[i_ids[n_count]] = self.bi[i_ids[n_count]] - gamma * gSSE_gBi 
            
            # 打印部分训练信息，反应代码当前跑的进度，但不能打的太多，会很卡很乱
            if n_round%10==0 and n_count%5000==0:
                print("n_round = %d, n_count = %d" % (n_round, n_count))
                print(self.P[uids[n_count],:])
                print(self.Q[:,i_ids[n_count]])
                print(self.bu[uids[n_count]])
                print(self.bi[i_ids[n_count]])
    
    print ("SVD trained") 
    
    # 先确定目标函数，目的是求最优解，故需要求对各个参数的偏导数，通过沿负梯度下降的方法来使目标函数收敛到最优值（应该是局部最优值）
    # 求偏导 + 更新迭代参数 ，这两部可以融合在一起，但分开更清晰

  def pred_SVD(self, uid, i_id):
    #根据当前参数，预测用户uid对Item（i_id）的打分  
    # 评分 = 均值 + 活动属性偏差 + 用户属性偏差 + 用户和活动的关联属性
    ans=self.mu + self.bi[i_id] + self.bu[uid] + np.dot(self.P[uid,:],self.Q[:,i_id])  
        
    #将打分范围控制在0-1之间
    if ans>1:  
        return 1  
    elif ans<0:  
        return 0
    return ans

  def sim_cal_UserCF(self, uid1, uid2 ):
    # uid1 和 uid2 是两个user重编码的序号，不是真正的ID
    similarity = 0.0
    
    # 计算两个用户之间相似度时直接查计算好的相似度矩阵，并返回相应值
    #userSimMatrix_users = sio.mmread("userSimMatrix_userBased").todense()
    # 原本是直接通过读取相似度矩阵并查询，后来代码跑的时间很长，原因是这个函数调用的次数非常频繁，但如果每次都重新读取矩阵文件，再
    # 将数据放入缓存，会造成很多时间消耗，而且缓存不释放，占用电脑内存，导致后面代码跑了一个多小时，只生成了81条数据就不动了
    # 优化：在初始化的时候读取一次文件，并保存在缓存中，每次查询缓存里的数据即可
    
    similarity = self.userSimMatrix_users[uid1, uid2]
    #print("sim_cal_UserCF similarity = %d"%similarity)  
    return similarity  

  def userCFReco(self, userId, eventId):
    """
    根据User-based协同过滤，得到event的推荐度
    基本的伪代码思路如下：
    for item i
      for every other user v that has a preference for i
        compute similarity s between u and v
        incorporate v's preference for i weighted by s into running average
    return top items ranked by weighted average
    """
    
    ans = 0.0

    u = self.userIndex[userId]
    i = self.eventIndex[eventId]
    # 获取参加该活动的所有用户（相关联用户协助预测）
    # 如果关联用户很多，可以考虑设置相似度的阈值或邻居人数，选取部分用户计算推荐度
    relevant_users = self.usersForItem[i]
    # 公式中的分子和分母（计算预测公式）
    numerator_1 = 0.0
    denominator_1 = 0.0
    # 计算涉及到的 用户1和用户2的相似度、用户2相对于平均打分的偏置 在上面已计算完成，直接调用
    for user0 in relevant_users:
        numerator_1 = numerator_1 + self.sim_cal_UserCF(u, user0) * row_userEventScores[user0,i]
        denominator_1 = denominator_1 + self.sim_cal_UserCF(u, user0)
     
    #增加对分母的判断，避免分母为0的情况，如果分母为0说明两者相似度系数为0
    if denominator_1==0:
        ans = 0
    else:
        ans = mean_score[u] + numerator_1 / denominator_1
        
    #print("UserCFReco ans = %d"%ans)  
    return ans


  def sim_cal_ItemCF(self, i_id1, i_id2):
        
    #计算Item i_id1和i_id2之间的相似性
    # i_id1 和 i_id2 是重编码的活动序号，不是原始ID

    similarity = 0.0

    #userSimMatrix_events = sio.mmread("eventSimMatrix_eventBased").todense()
    # 上面读文件的代码挖的坑同用户协同过滤一样，不能这样做
    similarity = self.userSimMatrix_events[i_id1, i_id2]
    #print("sim_cal_ItemCF similarity = %d"%similarity)  
    return similarity     
            
  def eventCFReco(self, userId, eventId):    
    """
    根据基于物品的协同过滤，得到Event的推荐度
    基本的伪代码思路如下：
    for item i 
        for every item j tht u has a preference for
            compute similarity s between i and j
            add u's preference for j weighted by s to a running average
    return top items, ranked by weighted average
    """
    ans = 0.0

    u = self.userIndex[userId]
    i = self.eventIndex[eventId]
    # 获取该用户参加的所有活动（相关联活动协助预测）
    relevant_events = self.itemsForUser[u]
    # 预测公式中的分子和分母
    numerator_2 = 0.0
    denominator_2 = 0.0
    # 计算涉及到的 活动1和用活动2的相似度等参数 在上面已计算完成，直接调用
    # 注意，这里的分子是活动的相似度 乘以 用户对该活动的打分，而不是（打分-均值），这个跟上面基于用户的协同过滤得到推荐度有些差别
    # 我选择整行的活动一起计算，因为无关的活动相似度为0，关系不紧密的活动，相似度
    # 加权值也会比较小，所以应该影响不大
    for event0 in relevant_events:
        numerator_2 = numerator_2 + self.sim_cal_ItemCF(i, event0) * self.userEventScores[u,event0]
        denominator_2 = denominator_2 + self.sim_cal_ItemCF(i, event0)
    
    # 规避分母为0的情况
    if denominator_2==0:
        ans = 0
    else:    
        ans = numerator_2 / denominator_2
        
    #print("eventCFReco ans = %d"%ans)  
    return ans
    
  def svdCFReco(self, userId, eventId):
    #基于模型的协同过滤, SVD++/LFM
    u = self.userIndex[userId]
    i = self.eventIndex[eventId]

    return self.pred_SVD(u,i)

  def userReco(self, userId, eventId):
    """
    类似基于User-based协同过滤，只是用户之间的相似度由用户本身的属性得到，计算event的推荐度
    基本的伪代码思路如下：
    for item i
      for every other user v that has a preference for i
        compute similarity s between u and v
        incorporate v's preference for i weighted by s into running aversge
    return top items ranked by weighted average
    """
    i = self.userIndex[userId]
    j = self.eventIndex[eventId]

    vs = self.userEventScores[:, j]
    sims = self.userSimMatrix[i, :]

    prod = sims * vs

    try:
      return prod[0, 0] - self.userEventScores[i, j]
    except IndexError:
      return 0

  def eventReco(self, userId, eventId):
    """
    类似基于Item-based协同过滤，只是item之间的相似度由item本身的属性得到，计算Event的推荐度
    基本的伪代码思路如下：
    for item i 
      for every item j that u has a preference for
        compute similarity s between i and j
        add u's preference for j weighted by s to a running average
    return top items, ranked by weighted average
    """
    i = self.userIndex[userId]
    j = self.eventIndex[eventId]
    js = self.userEventScores[i, :]
    psim = self.eventPropSim[:, j]
    csim = self.eventContSim[:, j]
    pprod = js * psim
    cprod = js * csim
    
    pscore = 0
    cscore = 0
    try:
      pscore = pprod[0, 0] - self.userEventScores[i, j]
    except IndexError:
      pass
    try:
      cscore = cprod[0, 0] - self.userEventScores[i, j]
    except IndexError:
      pass
    return pscore, cscore

  def userPop(self, userId):
    """
    基于用户的朋友个数来推断用户的社交程度
    主要的考量是如果用户的朋友非常多，可能会更倾向于参加各种社交活动
    """
    if userId in self.userIndex:
      i = self.userIndex[userId]
      try:
        return self.numFriends[0, i]
      except IndexError:
        return 0
    else:
      return 0

  def friendInfluence(self, userId):
    """
    朋友对用户的影响
    主要考虑用户所有的朋友中，有多少是非常喜欢参加各种社交活动/event的
    用户的朋友圈如果都积极参与各种event，可能会对当前用户有一定的影响
    """
    nusers = np.shape(self.userFriends)[1]
    i = self.userIndex[userId]
    return (self.userFriends[i, :].sum(axis=0) / nusers)[0,0]

  def eventPop(self, eventId):
    """
    本活动本身的热度
    主要是通过参与的人数来界定的
    """
    i = self.eventIndex[eventId]
    return self.eventPopularity[i, 0]



In [11]:
def generateRSData(RS, train=True, header=True):
    """
    把前面user-based协同过滤 和 item-based协同过滤，以及各种热度和影响度作为特征组合在一起
    生成新的训练数据，用于分类器分类使用
    """
    fn = "train.csv" if train else "test.csv"
    fin = open(fn, 'r')
    fout = open("RS_" + fn, 'w')
    
    #忽略第一行（列名字）
    fin.readline().strip().split(",")
    
    # write output header
    if header:
      ocolnames = ["invited", "userCF_reco", "evtCF_reco","svdCF_reco","user_reco", "evt_p_reco",
        "evt_c_reco", "user_pop", "frnd_infl", "evt_pop"]
      if train:
        ocolnames.append("interested")
        ocolnames.append("not_interested")
      fout.write(",".join(ocolnames) + "\n")
    
    ln = 0
    for line in fin:
      ln += 1
      if ln%500 == 0:
          print ("%s:%d (userId, eventId)=(%s, %s)" % (fn, ln, userId, eventId))
          #break;
      
      cols = line.strip().split(",")
      userId = cols[0]
      eventId = cols[1]
      invited = cols[2]
      
      userCF_reco = RS.userCFReco(userId, eventId)
      itemCF_reco = RS.eventCFReco(userId, eventId)
      svdCF_reco = RS.svdCFReco(userId, eventId)
        
      user_reco = RS.userReco(userId, eventId)
      evt_p_reco, evt_c_reco = RS.eventReco(userId, eventId)
      user_pop = RS.userPop(userId)
     
      frnd_infl = RS.friendInfluence(userId)
      evt_pop = RS.eventPop(eventId)
      ocols = [invited, userCF_reco, itemCF_reco, svdCF_reco,user_reco, evt_p_reco,
        evt_c_reco, user_pop, frnd_infl, evt_pop]
      
      if train:
        ocols.append(cols[4]) # interested
        ocols.append(cols[5]) # not_interested
      fout.write(",".join(map(lambda x: str(x), ocols)) + "\n")
    
    fin.close()
    fout.close()


In [12]:
RS = RecommonderSystem()
print ("生成训练数据...\n")
generateRSData(RS,train=True,  header=True)

print ("生成预测数据...\n")
generateRSData(RS, train=False, header=True)

SVD Train...
n_round = 0, n_count = 0
[0.08387341 0.00764184 0.18967457 0.36717707 0.32978268 0.26320094
 0.2524052  0.32860424 0.36969414 0.40404306 0.33787495 0.25839888
 0.16233282 0.240746   0.15007781 0.18397874 0.3429503  0.14587543
 0.27961638 0.0084594 ]
[0.33902048 0.41535043 0.20392093 0.37908884 0.03614334 0.21029504
 0.2158121  0.35005179 0.0177824  0.12527124 0.34852902 0.41850682
 0.27026183 0.06534529 0.35433595 0.03040684 0.35947218 0.35630205
 0.23963924 0.02939353]
-0.04
-0.04
n_round = 0, n_count = 5000
[ 0.40884093  0.14445671  0.16743264  0.06706455  0.31215671 -0.00091337
  0.19685357  0.23720289  0.02571676  0.21753718  0.08901758  0.26400879
  0.21370744  0.20000785  0.14633433  0.12323477  0.29431092  0.34773299
  0.38058358  0.16211579]
[0.02032378 0.23795501 0.13510514 0.38714766 0.33492045 0.05128374
 0.1559093  0.40607224 0.37415957 0.03090713 0.1711644  0.26187507
 0.1940971  0.27872118 0.03041138 0.27388829 0.16960878 0.06738418
 0.25340429 0.20102576]
-0

n_round = 30, n_count = 15000
[ 0.11987733  0.1897295   0.06992008  0.16088493  0.06313843  0.08775223
  0.13037113  0.1185874  -0.03243466  0.04151125  0.19364084  0.07483658
  0.066461    0.08497     0.04487639  0.13802234  0.14344927  0.07173877
  0.11299174 -0.08294418]
[ 0.02314919 -0.00445632  0.02304413  0.13474135  0.2520791   0.28484883
  0.27682671  0.00960613  0.22803877  0.08441104 -0.03901948 -0.00823336
  0.08830372  0.06098186  0.00260084 -0.03626562  0.25867787  0.11795012
  0.0503015   0.29646257]
-0.036835150770831916
-0.2655325726376718
n_round = 40, n_count = 0
[-0.01939991 -0.09010556  0.02356377  0.03170321  0.02782507  0.00096918
  0.03438786  0.04420995  0.0606652   0.08866271  0.00835139  0.01068668
  0.03296034 -0.00999031 -0.01851693  0.01989972  0.04212044 -0.01633555
  0.00940904 -0.00157668]
[ 0.21518744  0.24871215  0.09869291  0.1931606  -0.00135041  0.10468494
  0.09978071  0.20601851 -0.03209788  0.03678675  0.2069401   0.22190124
  0.13275399  0.03259

n_round = 70, n_count = 10000
[ 1.09739046e-02  2.50315819e-03 -1.21096433e-02  4.07702434e-05
  1.96473890e-03 -1.79963122e-03 -1.69443531e-03 -6.81622048e-03
  7.00447244e-03 -1.19966547e-02 -8.51307324e-03 -2.72056919e-02
  1.52682152e-03 -1.03663353e-02 -1.47951623e-02  5.35779000e-03
 -3.17238333e-03 -2.07139472e-02 -1.80742338e-02  6.46636126e-03]
[ 0.2833249   0.25111537  0.1268725   0.21607766  0.15889369  0.20763403
  0.24482216  0.09174626  0.15037129 -0.00383946  0.17330599  0.04873247
  0.21975395  0.11754968  0.13854655  0.19189976  0.15839582  0.04348844
  0.07083059  0.10377147]
-0.013815225842208031
0.6408588590266506
n_round = 70, n_count = 15000
[ 0.09676808  0.09191823  0.0471461   0.08003374  0.01909983 -0.00107381
  0.05676405  0.0487354  -0.03398455  0.0167753   0.10678084  0.04988001
  0.02490725  0.0002001   0.01250734  0.05795576  0.03773648  0.05861442
  0.09295289 -0.05418984]
[ 0.00565206 -0.01871601  0.01149066  0.09282297  0.19376071  0.22003394
  0.207673

test.csv:7500 (userId, eventId)=(3199685636, 1776393554)
test.csv:8000 (userId, eventId)=(3393388475, 680270887)
test.csv:8500 (userId, eventId)=(3601169721, 154434302)
test.csv:9000 (userId, eventId)=(3828963415, 3067222491)
test.csv:9500 (userId, eventId)=(4018723397, 2522610844)
test.csv:10000 (userId, eventId)=(4180064266, 2658555390)


时间、地点等特征都没有处理了，可以考虑用户看到event的时间与event开始时间的差、用户地点和event地点的差异。。。

1、基于用户和活动的协同过滤只是利用k近邻的方法，保存了原本的数据，当测试新数据时进行比对和计算，训练快，测试慢  
2、基于模型的协同过滤是训练了模型，然后可以抛弃原始训练数据，训练慢，测试快——线上的系统应该用这个更合理一些  
3、SVD分解应用很多  
    推荐系统中用到的SVD分解称为：隐因子分解  
    PCA降维里面用到SVD分解称为：主成分分析  
    语言文本用到SVD分解称为：隐性语义分析  

将生成的数据（修炼成果）读进来检查下有木有问题

In [13]:
import pandas as pd

new_train_data = pd.read_csv("RS_train.csv")
new_train_data.head()

,invited,userCF_reco,evtCF_reco,svdCF_reco,user_reco,evt_p_reco,evt_c_reco,user_pop,frnd_infl,evt_pop,interested,not_interested
0,0,[[-3.08693188e-05]],0.0,0.010140,0.0,0.0,0.0,0.001099,0.0,0.000018,0,0
1,0,[[-1.35525272e-20]],0.0,0.041679,0.0,0.0,0.0,0.001099,0.0,-0.000012,0,0
2,0,[[1.00000882]],1.0,0.516199,0.0,-1.0,-1.0,0.001099,0.0,0.000021,1,0
3,0,[[0.]],0.0,0.000000,0.0,0.0,0.0,0.001099,0.0,0.000041,0,0
4,0,[[0.25293482]],0.0,0.203597,0.0,0.0,0.0,0.001099,0.0,-0.000099,0,0


In [14]:
new_test_data = pd.read_csv("RS_test.csv")
new_test_data.head()

,invited,userCF_reco,evtCF_reco,svdCF_reco,user_reco,evt_p_reco,evt_c_reco,user_pop,frnd_infl,evt_pop
0,0,0,0,1.0,0.0,0.0,0.0,0.000199,0.0,0.000051
1,0,0,0,1.0,0.0,0.0,0.0,0.000199,0.0,0.000035
2,0,0,0,1.0,0.0,0.0,0.0,0.000199,0.0,0.000115
3,0,0,0,1.0,0.0,0.0,0.0,0.000199,0.0,0.000025
4,0,0,0,1.0,0.0,0.0,0.0,0.000199,0.0,-0.000014


发现训训练集和测试集生成的数据还是比较正常的，训练集中的userCF_reco这列数据格式有点问题，显示的是一个二维数组，但值确实只有一个，可能是上面处理的时候没有进行转化